In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from typing import List
from tqdm import tqdm
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost

import networkx
from networkx.algorithms.traversal.depth_first_search import dfs_edges

import warnings
warnings.filterwarnings('ignore')

/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_74642/1596029284.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
data = dict()
for dirname, _, filenames in os.walk('../kaggle/input'):
    for filename in filenames:
        table_name = filename.split('.')[0]
        table_path = os.path.join(dirname, filename)
        try:
            data[table_name] = pd.read_csv(table_path)
        except UnicodeDecodeError:
            data[table_name] = pd.read_csv(table_path, encoding='cp1252')
        except Exception as e:
            print(f"Error with {filename}: {e}")

# Split dict of dataframes by gender and other (supplemental) data
mens_data = dict()
womens_data = dict()
supplemental_data = dict()

for k, v in data.items():
    if k.startswith("M"):
        mens_data[k] = v
    elif k.startswith("W"):
        womens_data[k] = v
    else:
        supplemental_data[k] = v
        

In [3]:
def get_season_stats(dataset, detailed=False, post_season=False, year=2024):
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if detailed:
        if post_season:
            df = dataset[f"{gender}NCAATourneyDetailedResults"]
        else:
            df = dataset[f"{gender}RegularSeasonDetailedResults"]
        
    else:
        if post_season:
            df = dataset[f"{gender}NCAATourneyCompactResults"]
        else:
            df = dataset[f"{gender}RegularSeasonCompactResults"]
        
    df = df[df["Season"] == year]
    return df, gender

def compute_margins_of_victory(df):
    df["margin"] = df["WScore"] - df["LScore"]
    
    win_df = df[["WTeamID", "margin"]].rename(columns={"WTeamID": "TeamID"})
    lose_df = df[["LTeamID", "margin"]].rename(columns={"LTeamID": "TeamID"})
    lose_df["margin"] = -lose_df["margin"]

    res = pd.concat([win_df, lose_df], axis=0)
    return res.groupby("TeamID")["margin"].mean()

def join_team_names(df, data, gender="M"):
    """
    df: pd.DataFrame
        dataframe appending teams to
    data: dict[str, pd.DataFrame]
        dictionary of all table names and data
    """
    res = pd.merge(df, data[f"{gender}Teams"][["TeamID", "TeamName"]], on="TeamID")
    return res

def create_srs(df,gender):

    df["margin"] = df["WScore"] - df["LScore"]
    win_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"WTeamID": "team_id", "LTeamID": "opp_id"}
    )
    lose_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"LTeamID": "team_id", "WTeamID": "opp_id"}
    )
    lose_df["margin"] = -lose_df["margin"]

    teams = pd.concat([win_df, lose_df], axis=0)
    spreads = compute_margins_of_victory(df)
    
    terms = []
    solutions = []

    for team_id in spreads.keys():
        row = []
        opps = list(teams[teams["team_id"] == team_id]["opp_id"])

        for opp_id in spreads.keys():
            if opp_id == team_id:
                # coef for the team itself should be 1
                row.append(1)
            elif opp_id in opps:
                # coef for opponents is 1 over num of opps
                row.append(-1.0/len(opps))
            else:
                # teams not faced get a 0 coef
                row.append(0)
        terms.append(row)

        solutions.append(spreads[team_id])

    solutions, _, _, _ = np.linalg.lstsq(np.array(terms), np.array(solutions), rcond=None)
    
    ratings = list(zip( spreads.keys(), solutions ))
    srs = pd.DataFrame(ratings, columns=['team', 'rating'])
    rankings = srs.sort_values('rating', ascending=False).reset_index()[['team', 'rating']]
    rankings = join_team_names(rankings.rename(columns={"team": "TeamID"}), data, gender=gender)
    return rankings

def get_coach_win_perc(
    dataset: dict,
    regular_season: bool,
    year:int = 2024
) -> pd.DataFrame:
    """
    
    parameters
    ----------
    dataset: dict
        dictionary of datasets to use. it will be
        mens_data or womens_data.
        
    year: int
        year to filter data. it will get coaches stats for everything
        up until this year. (model can't have any look ahead bias). for post
        season games, use a year one less than the year of interest.
        
    returns
    -------
    coaches_stats: pd.DataFrame
        dataframe with count of wins, win percentage, and std dev
        of wins.
    """
    
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if regular_season:
        df = dataset[f"{gender}RegularSeasonCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] <= year]
    else:
        df = dataset[f"{gender}NCAATourneyCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] < year]
        
    
    
    winning_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "WTeamID"],
        right_on=["Season", "TeamID"]
    )

    winning_coaches_df = winning_coaches_df[
        (winning_coaches_df['DayNum'] >= winning_coaches_df['FirstDayNum']) 
        & (winning_coaches_df['DayNum'] <= winning_coaches_df['LastDayNum'])
    ]
    winning_coaches_df["win"] = 1

    #Make sure the join dind't create dupes
    assert len(winning_coaches_df) == len(df)

    losing_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "LTeamID"],
        right_on=["Season", "TeamID"]
    )

    losing_coaches_df = losing_coaches_df[
        (losing_coaches_df['DayNum'] >= losing_coaches_df['FirstDayNum']) 
        & (losing_coaches_df['DayNum'] <= losing_coaches_df['LastDayNum'])
    ]
    losing_coaches_df["win"] = 0

    #Make sure the join dind't create dupes
    assert len(losing_coaches_df) == len(df)

    coaches_df = pd.concat(
        [
            losing_coaches_df[["CoachName", "win"]],
            winning_coaches_df[["CoachName", "win"]]
        ],
        axis=0
    )

    coach_stats = (
        coaches_df
        .groupby("CoachName")["win"]
        .describe()
        .sort_values("count", ascending=False)
        [["count", "mean", "std"]]
        .fillna(0)
    )

    return coach_stats
def get_system_ratings(
    mens_dataset, #There are only ratings for men
    systems: List[str],
    year: int=2024,
):
    """
    gets system ratings for each team for specified systems for a specific year.
    
    parameters
    ---------
    mens_dataset: dict
        dictionary of datasets for men
    systems: List[str]
        list of dictionaries we are interested in seeing
    year: int
        year to look for ratings
    moving_average: str
        specifies how to calculate rolling ratings for given systems.
        if None, the system takes the most recent system rating
    
    returns
    -------
    df: pd.DataFrame
        data that reflects ratings for a team
    """
    
    # Filter by season - only take most recent
    df = mens_dataset["MMasseyOrdinals"]
    df = df[df["Season"] == year]
    
    # Filter by system
    df = df[df["SystemName"].isin(systems)]
    
    latest_rank = (
        df
        .sort_values("RankingDayNum")
        .groupby(["TeamID","SystemName"])
        ["OrdinalRank"]
        .last()
        .unstack("SystemName")
        .reset_index().
        rename(columns = {i: i+"_latest" for i in systems})
    )
    
    transformed_df = (
        df
        .sort_values(by="RankingDayNum")
        .groupby(["TeamID", "SystemName"], group_keys=False)
        ["OrdinalRank"]
        .rolling(5) # TODO: Parameterize this (window and moving average method)
        .mean()
        .unstack("SystemName")
        .reset_index()
        .drop("level_1", axis=1)
        .groupby("TeamID")
        [systems]
        .last()
        .reset_index()
        .rename(columns = {i: i+"_rolling" for i in systems})
    )
    
    res = pd.merge(latest_rank, transformed_df, on="TeamID")

    return res

def get_post_season(data, year):
    
    df, gender = get_season_stats(
            data, 
            detailed=False, 
            post_season=True, 
            year=year
    )
    
    # Shuffle teams for positional encoding (model shouldn't have winning teams features as the same)
    df["TeamID"] = np.where(
        np.random.uniform(0,1, size=len(df)) > .5, 
        df["WTeamID"], 
        df["LTeamID"]
    )
    df["team_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["WScore"], 
        df["LScore"]
    )
    df["OppID"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LTeamID"], 
        df["WTeamID"]
    )
    df["opp_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LScore"], 
        df["WScore"]
    )
    df = df.drop(
        ["WTeamID", "LTeamID", "WScore", "LScore", "WLoc", "NumOT"],
        axis=1
    )
    
    return df

def get_features(mens_data, year, systems):
    # Season Stats
    df, gender = get_season_stats(
        mens_data, 
        detailed=False, 
        post_season=False, 
        year=year
    )

    # Rating System
    srs = create_srs(df, gender)

    # System Ratings
    system_ratings = get_system_ratings(
        mens_data, 
        systems=systems
    ) #KenPom, Nolan ELO, EPSN BPI

    # Ratings df
    ratings_df = pd.merge(
                srs,
                system_ratings,
                on="TeamID"
    )

    # Coaches postseason win stats
    coaches_postseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=False, 
        year=year
    ).rename(columns={"count": "count_post", "mean": "mean_post", "std": "std_post"})

    # Coaches regular season win stats
    coaches_regseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=True, 
        year=year
    ).rename(columns={"count": "count_reg", "mean": "mean_reg", "std": "std_reg"})

    coaches_df = pd.merge(
        coaches_regseason_win_df,
        coaches_postseason_win_df,
        on="CoachName",
        how="left"
    ).fillna(0)

    # Get coaches for the year and only grab the most recent coach for a certain team
    curr_coaches = (
        mens_data["MTeamCoaches"][
            mens_data["MTeamCoaches"]["Season"] == year
        ]
        .sort_values("FirstDayNum")
        .groupby("TeamID")["CoachName"]
        .last()
        .reset_index()
    )

    # Get coach stats for current coaches
    coaches_df = pd.merge(
        curr_coaches,
        coaches_df,
        on="CoachName",
        how="left"
    )


    feature_df = (
        pd.merge(
            ratings_df,
            coaches_df
        )
        .drop(["TeamName", "CoachName"], axis=1)
    )

    
    return feature_df


def merge_features_to_games(feature_df, post_season_df, year, training=True):
    
    post_season_merged = pd.merge(
        pd.merge(
            feature_df,
            post_season_df,
            on="TeamID",
        ),
        feature_df,
        left_on="OppID",
        right_on="TeamID",
        suffixes=("_team", "_opp")
    )
    if training:
        post_season_merged["win"] = post_season_merged["team_score"] > post_season_merged["opp_score"]
        post_season_merged = (
            post_season_merged
            .drop(
                ["TeamID_team", "team_score", "OppID", "TeamID_opp", "opp_score", "DayNum"], 
                axis=1
            )
        )
    return post_season_merged

In [4]:
mens_data.keys()

dict_keys(['MNCAATourneyDetailedResults', 'MNCAATourneyCompactResults', 'MSeasons', 'MRegularSeasonDetailedResults', 'MNCAATourneySlots', 'MGameCities', 'MConferenceTourneyGames', 'MRegularSeasonCompactResults', 'MNCAATourneySeedRoundSlots', 'MTeamConferences', 'MTeamCoaches', 'MMasseyOrdinals', 'MTeams', 'MNCAATourneySeeds', 'MSecondaryTourneyTeams', 'MTeamSpellings', 'MSecondaryTourneyCompactResults'])

In [153]:
df = mens_data["MRegularSeasonDetailedResults"]
df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [194]:
df_2023, _ = get_season_stats(
    mens_data,
    detailed=True,
    post_season=False,
    year=2023
)

In [241]:
def get_team_stats(df):

    df["margin"] = df["WScore"] - df["LScore"]
    win_df = df.rename(
        columns={"WTeamID": "team_id", "LTeamID": "opp_id", "WLoc": "Loc"}
    )
    win_df = win_df.rename(columns={col: col[1:] + "_opp" for col in win_df.columns if col.startswith("L") and col != "Loc"})
    win_df = win_df.rename(columns={col: col[1:] for col in win_df.columns if col.startswith("W") and not col.endswith("_opp")})
    
    lose_df = df.rename(
        columns={"LTeamID": "team_id", "WTeamID": "opp_id", "WLoc": "Loc"}
    )
    lose_df = lose_df.rename(columns={col: col[1:] for col in lose_df.columns if col.startswith("L") and col != "Loc"})
    lose_df = lose_df.rename(columns={col: col[1:] + "_opp" for col in lose_df.columns if col.startswith("W")})
    lose_df["Loc"] = lose_df["Loc"].apply(lambda x: "H" if x == "A" else "A" if x == "H" else "N")
    lose_df["margin"] = -lose_df["margin"]

    teams = pd.concat([win_df, lose_df], axis=0)
    return teams

In [242]:
df = get_team_stats(df_2023)

In [255]:
df.groupby(["Season", "team_id"])[
    ['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast',
       'TO', 'Stl', 'Blk', 'PF', 'FGM_opp', 'FGA_opp', 'FGM3_opp', 'FGA3_opp',
       'FTM_opp', 'FTA_opp', 'OR_opp', 'DR_opp', 'Ast_opp', 'TO_opp',
       'Stl_opp', 'Blk_opp', 'PF_opp', 'margin'
    ]
].describe()

FGM                                                       \
               count       mean       std   min    25%   50%    75%   max   
Season team_id                                                              
2023   1101     26.0  25.384615  4.409779  15.0  22.25  25.5  28.00  34.0   
       1102     32.0  23.625000  4.217858  15.0  20.75  24.0  26.00  35.0   
       1103     31.0  25.387097  5.083814  14.0  23.00  26.0  28.50  36.0   
       1104     34.0  27.764706  4.942450  17.0  25.25  27.0  29.00  39.0   
       1105     30.0  24.233333  3.856775  17.0  22.00  24.5  27.00  32.0   
...              ...        ...       ...   ...    ...   ...    ...   ...   
       1473     29.0  24.206897  5.052434  15.0  21.00  23.0  29.00  35.0   
       1474     30.0  25.666667  4.443774  18.0  22.50  26.0  28.00  39.0   
       1475     29.0  26.172414  3.036316  19.0  24.00  26.0  28.00  32.0   
       1476     30.0  22.900000  5.504544  15.0  18.25  21.5  26.75  34.0   
       1477     32.0  25.593750  3.599815  19.0  23.00  26.0  28.00  31.0   

                 FGA             ... PF_opp       margin             \
               count       mean  ...    75%   max  count       mean   
Season team_id                   ...                                  
2023   1101     26.0  58.346154  ...  22.75  28.0   26.0  -3.692308   
       1102     32.0  51.125000  ...  20.00  29.0   32.0  -0.125000   
       1103     31.0  57.451613  ...  21.00  27.0   31.0   5.838710   
       1104     34.0  62.205882  ...  23.00  29.0   34.0  13.676471   
       1105     30.0  56.500000  ...  21.00  25.0   30.0  -3.066667   
...              ...        ...  ...    ...   ...    ...        ...   
       1473     29.0  57.172414  ...  17.00  29.0   29.0  -8.448276   
       1474     30.0  58.933333  ...  20.00  27.0   30.0   0.100000   
       1475     29.0  61.310345  ...  21.00  30.0   29.0  -0.931034   
       1476     30.0  52.433333  ...  19.75  23.0   30.0  -4.266667   
       1477     32.0  56.687500  ...  18.25  22.0   32.0  -3.562500   

                                                           
                      std   min    25%   50%    75%   max  
Season team_id                                             
2023   1101     12.970025 -30.0 -14.00  -5.5   6.00  19.0  
       1102     14.013243 -30.0  -8.50  -3.0   8.75  28.0  
       1103     15.701585 -21.0  -6.50   6.0  17.50  37.0  
       1104     17.705881 -24.0   3.00  11.0  21.75  57.0  
       1105     12.926672 -31.0 -11.00  -6.0   4.75  34.0  
...                   ...   ...    ...   ...    ...   ...  
       1473     14.044791 -33.0 -17.00  -9.0   1.00  18.0  
       1474      9.444393 -20.0  -6.75   0.0   6.00  29.0  
       1475     11.186507 -24.0  -8.00  -3.0  10.00  18.0  
       1476     11.913288 -31.0 -11.00  -5.0   4.75  15.0  
       1477      9.055162 -25.0  -8.25  -3.5   2.00  15.0  

[363 rows x 216 columns]

In [148]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# def get_advanced_stats(year):
    
#     ################################## TEAM ##########################################


#     # URL of the page you want to scrape
#     url = f'https://www.sports-reference.com/cbb/seasons/men/{year}-advanced-school-stats.html'

#     # Send a GET request to the webpage
#     response = requests.get(url)

#     # Parse the HTML content of the page
#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Find the table by ID or class
#     table = soup.find('table', {'id': 'adv_school_stats'})

#     # Extract the column headers from the <thead> part of the table
#     headers = [header.text for header in table.find('thead').find_all('th')]
#     headers = [
#         i for i in headers 
#         if (i != "") 
#         and (i != "\xa0") 
#         and (i not in ['School Advanced', 'Overall', 'Conf.', 'Home', 'Away', 'Points', 'Opponent Advanced', 'Rk'])
#     ]

#     # Extract rows from the <tbody> part of the table
#     rows = table.find('tbody').find_all('tr')

#     # Extract data from each row
#     data = []
#     for row in rows:
#         cols = row.find_all('td')
#         cols = [ele.text.strip() for ele in cols if ele.text.strip() != ""]
#         data.append(cols)  # Get rid of empty values

#     # Create the DataFrame
#     df = pd.DataFrame(data, columns=headers)  # headers[1:] because the first header is usually the rank or an empty string


#     ################################## OPP ##########################################

#     # URL of the page you want to scrape
#     url = f'https://www.sports-reference.com/cbb/seasons/men/{year}-advanced-opponent-stats.html'

#     # Send a GET request to the webpage
#     response = requests.get(url)

#     # Parse the HTML content of the page
#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Find the table by ID or class
#     table = soup.find('table', {'id': 'adv_opp_stats'})

#     # Extract the column headers from the <thead> part of the table
#     headers = [header.text for header in table.find('thead').find_all('th')]
#     headers = [
#         i for i in headers 
#         if (i != "") 
#         and (i != "\xa0") 
#         and (i not in ['Overall', 'Conf.', 'Home', 'Away', 'Points', 'Opponent Advanced', 'Rk'])
#     ]

#     # Extract rows from the <tbody> part of the table
#     rows = table.find('tbody').find_all('tr')

#     # Extract data from each row
#     data = []
#     for row in rows:
#         cols = row.find_all('td')
#         cols = [ele.text.strip() for ele in cols if ele.text.strip() != ""]
#         data.append(cols)  # Get rid of empty values

#     # Create the DataFrame
#     opp_df = pd.DataFrame(data, columns=headers)  # headers[1:] because the first header is usually the rank or an empty string

#     df = df.dropna().reset_index(drop=True)
#     opp_df = (
#         opp_df
#         .dropna()
#         .reset_index(drop=True)
#         .drop(["G", "W", "L", "W-L%", "SRS", "SOS", "W", "L", "W", "L", "W", "L", "Tm.", "Opp."], axis=1)
#         .rename(columns={col: col + "_opp" for col in opp_df.columns if col != "School"})
#     )
#     res = pd.merge(df, opp_df, on="School")

#     res["School"] = res["School"].str.replace("NCAA","").str.strip()

#     return res


# # Works 2010 and on
# advanced_stats = dict()
# for year in range(2010, 2025):
#     print(year)
#     advanced_stats[year] = get_advanced_stats(year)